In [1]:
%matplotlib inline
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from plots import *
from kls import *
from torch._six import inf
from torch.distributions.normal import Normal
from torch.distributions.multivariate_normal import MultivariateNormal as mvn
from torch.distributions.one_hot_categorical import OneHotCategorical as cat
from torch.distributions.gamma import Gamma
from torch import logsumexp
import sys
import time
import datetime

In [2]:
N = 50
D = 2
## Model Parameters
NUM_SAMPLES = 10
NUM_HIDDEN = 64
NUM_LATENTS = 2
NUM_OBS = D
BATCH_SIZE = 20
NUM_EPOCHS = 1000
LEARNING_RATE = 1e-3
CUDA = False
PATH = 'circles'

In [8]:
Xs = torch.from_numpy(np.load('circles/obs.npy')).float()
Xs = Xs.transpose(1,2)
mus_true = torch.from_numpy(np.load('circles/mus.npy')).float()
rads_true = torch.from_numpy(np.load('circles/rads.npy')).float()
num_seqs = Xs.shape[0]

In [34]:
# class Encoder(nn.Module):
#     def __init__(self, num_obs=D+D,
#                        num_stats=NUM_STATS,
#                        num_hidden=NUM_HIDDEN,
#                        num_latents=NUM_LATENTS):
#         super(self.__class__, self).__init__()
#         self.enc_stats = nn.Sequential(
#             nn.Linear(num_obs, num_hidden),
#             nn.ReLU(),
#             nn.Linear(num_hidden, num_stats))
#         self.enc_hidden = nn.Sequential(
#             nn.Linear(num_stats, num_hidden),
#             nn.ReLU())
#         self.mus_mean = nn.Sequential(
#             nn.Linear(num_hidden, num_latents))
#         self.mus_log_sigma = nn.Sequential(
#             nn.Linear(num_hidden, num_latents))
        
#     def forward(self, obs, K, D, num_samples, batch_size):
#         stats = self.enc_stats(obs).view(batch_size, N, -1).sum(1)
#         hidden = self.enc_hidden(stats)
#         q_mean = self.mus_mean(hidden).view(-1, K, D)
#         q_sigma = torch.exp(self.mus_log_sigma(hidden2).view(-1, K, D))
#         mus = Normal(q_mean, q_sigma).sample((num_samples,))  
#         return q_mean, q_sigma, mus
    
class Encoder(nn.Module):
    def __init__(self, num_obs=D+D,
                       num_hidden=NUM_HIDDEN,
                       num_latents=NUM_LATENTS):
        super(self.__class__, self).__init__()
        self.enc_hidden = nn.Sequential(
            nn.Linear(num_obs, num_hidden),
            nn.ReLU(),
            nn.Linear(num_hidden, num_hidden),
            nn.ReLU())
        self.local_mean = nn.Sequential(
            nn.Linear(num_hidden, num_latents))
        self.local_log_sigma = nn.Sequential(
            nn.Linear(num_hidden, num_latents))
        
    def forward(self, obs, N, D, batch_size):
        hidden = self.enc_hidden(obs)
        q_mean = self.local_mean(hidden).view(batch_size, N, D)
        q_sigma = torch.exp(self.local_log_sigma(hidden).view(batch_size, N, D))
        return q_mean, q_sigma
    
    
class Decoder(nn.Module):
    def __init__(self, num_obs=NUM_OBS,
                       num_hidden=NUM_HIDDEN,
                       num_latents=NUM_LATENTS+D):
        super(self.__class__, self).__init__()
        self.dec_hidden = nn.Sequential(
            nn.Linear(num_latents, num_hidden),
            nn.ReLU(),
            nn.Linear(num_hidden, num_hidden),
            nn.ReLU())
        self.obs_mean = nn.Sequential(
            nn.Linear(num_hidden, num_obs))
        self.obs_log_std = nn.Sequential(
            nn.Linear(num_hidden, num_obs))

    def forward(self, embed, obs, N, D, batch_size):
        hidden = self.dec_hidden(embed)           
        p_mean = self.obs_mean(hidden).view(batch_size, N, D)
        p_sigma = torch.exp(self.obs_log_std(hidden)).view(batch_size, N, D)
        ll_recon = Normal(p_mean, p_sigma).log_prob(obs).sum(-1).sum(-1)  
        return p_mean, p_sigma, ll_recon

In [35]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 1e-2)     
        
def initialize():
    enc = Encoder()
    dec = Decoder()
    optimizer =  torch.optim.Adam(list(enc.parameters()) + list(dec.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))    
    return enc, dec, optimizer
enc, dec, optimizer = initialize()

In [36]:
prior_mean = torch.zeros((BATCH_SIZE, N, D))
prior_sigma = torch.ones((BATCH_SIZE, N, D))

def reparam(q_mus, q_sigma, N, D, num_samples, batch_size):
    eps = Normal(torch.zeros((num_samples, batch_size, N, D)), torch.ones((num_samples, batch_size, N, D))).sample()
    ws = q_mus.unsqueeze(0).repeat(num_samples, 1, 1, 1) + torch.mul(q_sigma.unsqueeze(0).repeat(num_samples, 1, 1, 1), eps)
    log_qs = Normal(q_mus, q_sigma).log_prob(ws).sum(-1).sum(-1) ## S * B
    return ws, log_qs
    
    
def oneshot(x, mus, N, D, num_samples, batch_size):
    mus_expand = mus.unsqueeze(1).repeat(1, N, 1)
    data = torch.cat((x, mus_expand), dim=-1).view(batch_size*N, -1)
    q_mean, q_sigma = enc(data, N, D, batch_size)
    ws, log_qs = reparam(q_mean, q_sigma, N, D, num_samples, batch_size)
    log_p_joints = torch.zeros((num_samples, batch_size))
    for s in range(num_samples):
        log_pr = Normal(prior_mean, prior_sigma).log_prob(ws[s]).sum(-1).sum(-1)
        embed = torch.cat((ws[s], mus_expand), -1).view(batch_size*N, -1)
        p_mean, p_sigma, ll_recon = dec(embed, x, N, D, batch_size)
        log_p_joints[s] = ll_recon + log_pr
    log_weights = log_p_joints - log_qs
    elbo = log_weights.mean(0).mean()
    return elbo

def shuffler(batch_Xs, N, D, batch_size):
    indices = torch.cat([torch.randperm(N).unsqueeze(0) for b in range(batch_size)])
    indices_Xs = indices.unsqueeze(-1).repeat(1, 1, D)
    return torch.gather(batch_Xs, 1, indices_Xs)

In [ ]:
ELBOs = []

num_batches = int((Xs.shape[0] / BATCH_SIZE))
time_start = time.time()

for epoch in range(NUM_EPOCHS):
    indices = torch.randperm(num_seqs)
    ELBO = 0.0
    for step in range(num_batches):
        optimizer.zero_grad()
        batch_indices = indices[step*BATCH_SIZE : (step+1)*BATCH_SIZE]
        batch_Xs = Xs[batch_indices]
        batch_mus = mus_true[batch_indices]
        batch_Xs = shuffler(batch_Xs, N, D, BATCH_SIZE)
        elbo = oneshot(batch_Xs, batch_mus, N, D, NUM_SAMPLES, BATCH_SIZE)
        loss = - elbo
        loss.backward()
        optimizer.step()
        ELBO += elbo.item() 
    ELBO /= num_batches
    ELBOs.append(ELBO)
    
    if epoch%10 == 0:
        time_end = time.time()  
        print('epoch=%d, ELBO=%.3f (%ds)' % (epoch, ELBO, time_end - time_start))
        time_start = time.time()


epoch=0, ELBO=-149.244 (0s)
epoch=10, ELBO=-102.336 (0s)
epoch=20, ELBO=-82.893 (0s)
epoch=30, ELBO=-79.308 (0s)
epoch=40, ELBO=-78.629 (0s)
epoch=50, ELBO=-78.433 (0s)
epoch=60, ELBO=-78.416 (0s)
epoch=70, ELBO=-77.308 (0s)
epoch=80, ELBO=-77.809 (0s)
epoch=90, ELBO=-78.428 (0s)
epoch=100, ELBO=-77.703 (0s)
epoch=110, ELBO=-77.314 (0s)
epoch=120, ELBO=-78.328 (0s)
epoch=130, ELBO=-76.796 (0s)
epoch=140, ELBO=-76.985 (0s)
epoch=150, ELBO=-76.584 (0s)
epoch=160, ELBO=-76.940 (0s)
epoch=170, ELBO=-76.727 (0s)
epoch=180, ELBO=-76.340 (0s)
epoch=190, ELBO=-76.495 (0s)
epoch=200, ELBO=-76.982 (0s)
epoch=210, ELBO=-77.038 (0s)
epoch=220, ELBO=-76.372 (0s)


In [38]:
torch.save(enc_global.state_dict(), 'models/global-enc-' + PATH)
torch.save(enc_local.state_dict(), 'models/local-enc' + PATH)

NameError: name 'enc_global' is not defined

In [ ]:
def save_results(EUBOs, ELBOs, ESSs, KLs_eta_ex, KLs_eta_in, KL_z_ex, KL_z_in):
    fout = open('results/logs-' + PATH +'.txt', 'w+')
    fout.write('EUBOs, ELBOs, ESSs, KLs_eta_ex, KLs_eta_in, KL_z_ex, KL_z_in\n')
    for i in range(len(EUBOs)):
        fout.write(str(EUBOs[i]) + ', ' + str(ELBOs[i]) + ', ' + str(ESSs[i]) 
                   + str(KLs_eta_ex[i]) + str(KLs_eta_in[i]) + str(KLs_z_ex[i]) + str(KLs_z_in[i]) + '\n')
    fout.close()
save_results(EUBOs, ELBOs, ESSs, KLs_eta_ex, KLs_eta_in, KLs_z_ex, KLs_z_in)

In [ ]:
def plot_results(EUBOs, ELBOs, ESSs, KLs_eta_ex, KLs_eta_in, KLs_z_ex, KLs_z_in, num_samples):
    fig = plt.figure(figsize=(20, 30))
    ax1 = fig.add_subplot(3, 1, 1)
    ax1.plot(EUBOs, 'r', label='EUBOs')
    ax1.plot(ELBOs, 'b', label='ELBOs')
    ax1.tick_params(labelsize=18)
    ax1.set_ylim([-220, -130])
    ax1.legend(fontsize=18)
    ##
    ax2 = fig.add_subplot(3, 1, 2)
    ax2.plot(KLs_eta_ex, '#66b3ff', label='KLs_eta_ex')
    ax2.plot(KLs_eta_in, '#ff9999', label='KLs_eta_in')
    ax2.plot(KLs_z_ex, '#99ff99', label='KLs_z_ex')
    ax2.plot(KLs_z_in, 'gold', label='KLs_z_in')   
    ax2.plot(np.ones(len(KLs_z_in)) * 5, 'k', label='const=5.0')
    ax2.legend(fontsize=18)
    ax2.tick_params(labelsize=18)
    ax2.set_ylim([-1, 30])
    ##
    ax3 = fig.add_subplot(3, 1, 3)
    ax3.plot(np.array(ESSs) / num_samples, 'm', label='ESS')
    ax3.tick_params(labelsize=18)
    ax3.set_xlabel('epochs (%d gradient steps per epoch)'  % num_batches, size=18)
    ax3.legend()
    plt.savefig('results/train-' + PATH + '.svg')

In [ ]:
plot_results(EUBOs, ELBOs, ESSs, KLs_eta_ex, KLs_eta_in, KLs_z_ex, KLs_z_in, NUM_SAMPLES)

In [ ]:
STEPS = 50
def reverse(X, z, mus_prev, precisions_prev, N, D, K, batch_size):
    data = torch.cat((X, z), dim=-1).view(batch_size*N, -1)
    q_mean, q_nu, q_alpha, q_beta, q_sigma, mus, precisions = enc_global(data, K, D, 1, batch_size)  
    log_q_eta =  Normal(q_mean[0], q_sigma[0]).log_prob(mus_prev).sum(-1).sum(-1) + Gamma(q_alpha, q_beta).log_prob(precisions_prev).sum(-1).sum(-1)## B
    return log_q_eta

def test(x, Pi, N, K, D, num_samples, steps, batch_size):
    log_increment_weights = torch.zeros((steps, num_samples, batch_size))
    log_p_joints = torch.zeros((steps, num_samples, batch_size))
    log_qf = torch.zeros((steps-1, num_samples, batch_size))
    log_qr = torch.zeros((steps-1, num_samples, batch_size))
    Z_samples = torch.zeros((num_samples, batch_size, N, K))
    mus_prevs = torch.zeros((num_samples, batch_size, K, D))
    precisions_prevs = torch.zeros((num_samples, batch_size, K, D))
    
    for m in range(steps):
        if m == 0:
            for l in range(num_samples):
                q_mean, q_nu, q_alpha, q_beta, q_sigma, mus, precisions, log_q_eta = Init_step(x, N, D, K, batch_size)
                mus_prevs[l] = mus
                precisions_prevs[l] = precisions
                zs_pi, z, log_q_z = E_step(x, mus, precisions, N, D, K, batch_size)
                Z_samples[l] = z
                labels = z.nonzero()
                log_p_z = cat(Pi).log_prob(z).sum(-1)
                sigmas = 1. / torch.sqrt(precisions)
                log_p_x = Normal(mus[labels[:, 0], labels[:, -1], :].view(batch_size, N, D), sigmas[labels[:, 0], labels[:, -1], :].view(batch_size, N, D)).log_prob(x).sum(-1).sum(-1)
                log_increment_weights[m, l] = log_p_x + log_p_z - log_q_z     
                log_p_joints[m, l] = log_joints_gmm(x, z, Pi, mus, precisions, N, D, K, prior_mean, prior_nu, prior_alpha, prior_beta, batch_size)
                
        else:
            for l in range(num_samples):
                z_prev = Z_samples[l]
                q_mean, q_nu, q_alpha, q_beta, q_sigma, mus, precisions, log_q_eta = M_step(x, z_prev, N, D, K, batch_size)
                zs_pi, z, log_q_z = E_step(x, mus, precisions, N, D, K, batch_size)
                Z_samples[l] = z
                log_p_joints[m, l] = log_joints_gmm(x, z, Pi, mus, precisions, N, D, K, prior_mean, prior_nu, prior_alpha, prior_beta, batch_size)
                log_qf[m-1, l] = log_q_eta + log_q_z
                
                mus_prev = mus_prevs[l]
                precisions_prev = precisions_prevs[l]
                
                log_qr[m-1, l] = reverse(x, z, mus_prev, precisions_prev, N, D, K, batch_size)
                log_p_joint = log_joints_gmm(x, z, Pi, mus, precisions, N, D, K, prior_mean, prior_nu, prior_alpha, prior_beta, batch_size)
                log_increment_weights[m, l] = log_p_joint - log_q_z - log_q_eta
                mus_prevs[l] = mus
                precisions_prevs[l] = precisions
    detail_balances = log_p_joints[1:] - log_p_joints[:-1] -log_qf + log_qr
    increment_weights = torch.exp(log_increment_weights - logsumexp(log_increment_weights, 1).unsqueeze(1).repeat(1, num_samples, 1)).detach()
    esses = (1./ (increment_weights ** 2).sum(1))                   
    log_last_weights = log_increment_weights[-1] ## S * B
    ## EUBO and ELBO
    eubos = torch.mul(increment_weights, log_increment_weights).sum(1).mean(-1)
    elbos = log_increment_weights.mean(1).mean(-1)     
    return eubos, elbos, esses, log_increment_weights, detail_balances

In [ ]:
def sample_single_batch(num_seqs, N, K, D, batch_size):
    indices = torch.randperm(num_seqs)
    batch_indices = indices[0*batch_size : (0+1)*batch_size]
    batch_Xs = Xs[batch_indices]
    batch_Xs = shuffler(batch_Xs, N, K, D, batch_size)
    return batch_Xs
batch_Xs = sample_single_batch(num_seqs, N, K, D, BATCH_SIZE)
eubo, elbo, ess, log_increment_weights, detail_balances = test(batch_Xs, Pi, N, K, D, NUM_SAMPLES, STEPS, BATCH_SIZE)

In [ ]:

detail_balances.shape

In [ ]:
for i in range(BATCH_SIZE):
    log_weights = log_increment_weights[:, :, i]
    ess_stepwise = ess[:, i].data.numpy()
    fig = plt.figure(figsize=(20,6))
    ax1 = fig.add_subplot(1,3,1)
    ax2 = fig.add_subplot(1,3,2)
    ax3 = fig.add_subplot(1,3,3)
    weights = torch.exp(log_weights - logsumexp(log_weights, -1)[:, None]).data.numpy()
    db = detail_balances[:, :, i].mean(-1)
    ax1.plot(db.data.numpy(), 'r-o')
    ax1.set_ylim([-20, 20])
    for s in range(NUM_SAMPLES):
        ax2.plot(ess_stepwise, 'b-o')
        ax3.plot(weights[:, s], 'g-o')
        ax2.set_ylim([1.0, 7])


In [ ]:
ess

In [ ]:


def test(x, num_seqs, Pi, N, K, D, steps, batch_size):
    LLs = [] 
    for m in range(steps):
        if m == 0:
            mus, precisions, log_p_eta = inti_global(K, D, prior_mean, prior_nu, prior_alpha, prior_beta, batch_size)
            zs_pi, z, log_q_z = E_step(x, mus, precisions, N, D, K, batch_size)
        else:
            q_mean, q_nu, q_alpha, q_beta, q_sigma, mus, precisions, log_q_eta = M_step(x, z, N, D, K, batch_size)
            zs_pi, z, log_q_z = E_step(x, mus, precisions, N, D, K, batch_size)
            labels = z.nonzero()
            sigmas = 1. / torch.sqrt(precisions)
            ll = Normal(mus[labels[:, 0], labels[:, -1], :].view(batch_size, N, D), sigmas[labels[:, 0], labels[:, -1], :].view(batch_size, N, D)).log_prob(x).sum(-1).sum(-1).mean()
            LLs.append(ll.item())
    E_precisions = q_alpha / q_beta
    E_mus = q_mean
    E_z = torch.argmax(zs_pi, dim=-1)

    return z, mus, precisions, LLs, E_mus, E_precisions, E_z

x,z_true = sample_single_batch(num_seqs, N, K, D, BATCH_SIZE)
z, mus, precisions, LLs, E_mus, E_precisions, E_z = test(x, num_seqs, Pi, N, K, D, STEPS, BATCH_SIZE)

In [ ]:
def plot_final_samples(Xs, Zs, mus, precisions, steps, batch_size):
    colors = ['r', 'b', 'gold']
    fig = plt.figure(figsize=(25,100))
    for b in range(batch_size):
        ax = fig.add_subplot(int(batch_size / 5), 5, b+1)
        x = Xs[b].data.numpy()
        z = Zs[b].data.numpy()
        mu = mus[b].data.numpy()
        precision = precisions[b].data.numpy()

        covs = np.zeros((K, D, D))
        assignments = z
        for k in range(K):
            covs[k] = np.diag(1. / precision[k])
            xk = x[np.where(assignments == k)]
            ax.scatter(xk[:, 0], xk[:, 1], c=colors[k])
            plot_cov_ellipse(cov=covs[k], pos=mu[k], nstd=2, ax=ax, alpha=0.2, color=colors[k])
        ax.set_ylim([-10, 10])
        ax.set_xlim([-10, 10])
    plt.savefig('results/modes' + PATH + '.svg')

In [ ]:
plot_final_samples(x, E_z, E_mus, E_precisions, STEPS, BATCH_SIZE)

In [ ]:
plt.plot(LLs[:20])

In [ ]:
plt.plot(LLs)

In [ ]:
plt.plot(LLs[:10])

In [ ]:
np.array(LLs).mean()

In [ ]:
np.exp(-3)